# Elastic Stackの構築手順：インストール
----
Elasticsearchをインストールし、 各種パラメータを設定してクラスタを構築します。  
必要であればこの後のNotebookで利用するサンプルデータも投入します。

## 設定
アカウントや通信先などを設定します。

各サーバにAnsibleコマンドを発行するため、  
Ansibleを実行可能なユーザアカウントとそのSSHキーを次のセルで設定してください。

In [2]:
#elasticsearchをインストールするサーバーでansibleコマンドを実行するユーザー
USER='ansible'

#公開鍵認証を行う場合の秘密鍵のパス
#KEYPATH='/notebooks/etc/key/id_rsa'
KEYPATH = '~/.ssh/ansible_id_rsa'

この後のコマンドを実行するために、Client Nodeのアドレスか、それが無い場合はData Nodeのうちの1つのアドレスを設定してください。 

In [3]:
%env ES_CLIENT=ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com:9200

env: ES_CLIENT=ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com:9200


このNotebookを含むファイルを設置したディレクトリを読み込みます。  
※Logstash用の設定ファイルやサンプルデータはこの配下から取得します。変更する場合は下の設定を書き換えてください。

In [4]:
import os
notebooks_home = os.getcwd()
notebooks_home

'/notebooks'

## インストール

Elasticsearchのインストールし、設定、起動します。

### jdkのインストール
Elasticsearchを動作させるために、Javaをインストールします。


[00_Prerequisites for Literate Computing via Notebooks.ipynb](00_Prerequisites for Literate Computing via Notebooks.ipynb) の内容に従い、JVMの資材が**/tmp**ディレクトリに配置されているか確認して下さい。

In [5]:
!ls /tmp/jdk*

/tmp/jdk-8u112-linux-x64.rpm


次のコマンドを実行してインストールします。

In [6]:
!ansible-playbook playbooks/install_jdk.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [copy local rpm to remote] ************************************************
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

TASK [install jdk] *************************************************************
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [set JAVA_HOME] ***********************************************************
changed: [ec2-

### Elasticsearchのインストール
Elasticsearchのパッケージをインストールします。

次のコマンドを実行します。

In [7]:
!ansible-playbook playbooks/install_elasticsearch.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

TASK [install elasticsearch 5.0.0] *********************************************
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   
ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   
ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    un

### Elasticsearchの設定
Elasticsearchの設定を変更します。以下の2つを反映します。
- [01_03_Set_Inventory.ipynb](01_03_Set_Inventory.ipynb)で設定したhosts（インベントリ）のグループ
- [01_01_Outline.ipynb](01_01_Outline.ipynb)で出力したgroup_vars/all（パラメータ）

次のコマンドを実行して、elasticsearch.ymlを設定します。

In [8]:
!ansible-playbook playbooks/set_elasticsearch.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [set_cluster-name] ********************************************************
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

TASK [set_network-host] ********************************************************
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]

TASK [set_node-master] *********************************************************
changed: [ec2-

次のコマンドを実行して、ヒープサイズを設定します。/etc/elasticsearch/ にあるjvm.optionsにJavaの起動オプションとして指定します。

In [3]:
!ansible-playbook playbooks/set_heap-size.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

TASK [set -Xms jvm option] *****************************************************
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

TASK [set -Xmx jvm option] *****************************************************
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.am

次のコマンドを実行して、カーネルパラメータを変更します。  
/etc/sysctl.conf のvm.max_map_count に 262144 を設定しています。

In [10]:
!ansible-playbook playbooks/set_kernelparameter.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]

TASK [Set vm.max_map_count to 262144 in /etc/sysctl.conf] **********************
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   
ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   
ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    un

### Elasticsearchの起動
Elasticsearchを起動します。

次のコマンドを実行します。

In [11]:
!ansible-playbook playbooks/start_cluster.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [start elasticsearch] *****************************************************
changed: [ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]
changed: [ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   
ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   
ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    un

各サーバの起動は次のコマンドで確認してください。

In [12]:
!ansible all -m shell -a "sudo systemctl status elasticsearch |grep Active" -i ./hosts -u $USER --private-key=$KEYPATH

ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com | SUCCESS | rc=0 >>
   Active: active (running) since Thu 2017-03-16 07:15:52 UTC; 3s ago

ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com | SUCCESS | rc=0 >>
   Active: active (running) since Thu 2017-03-16 07:15:52 UTC; 3s ago

ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com | SUCCESS | rc=0 >>
   Active: active (running) since Thu 2017-03-16 07:15:52 UTC; 3s ago



次のように  
**Active: active (running)**    
と表示されれば正常に起動しています。

### Elasticsearchのテンプレートの投入
Shard数、Replica数を、インデックステンプレートの形でで投入することで設定します。

[Index Templates](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-templates.html)とはインデックス単位の設定やマッピングを定義したもので、インデックス生成時に適用されます。  
投入はAPIにて実施します。よってElasticsearchが起動したあとの投入となります。

次のコマンドを実行して、テンプレートを投入します。

In [13]:
!ansible-playbook playbooks/set_api_parameter.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [data-nodes[0]] ***********************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [post param] **************************************************************
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=0    unreachable=0    failed=0   



投入した内容は次のコマンドで表示できます。

In [14]:
!curl -XGET $ES_CLIENT/_template/template_base?pretty

{
  "template_base" : {
    "order" : 0,
    "template" : "*",
    "settings" : {
      "index" : {
        "number_of_shards" : "3",
        "number_of_replicas" : "1"
      }
    },
    "mappings" : { },
    "aliases" : { }
  }
}


- テンプレートには任意の名称を指定できます。このコマンドではtemplate_baseという名称にしています。
- **"template"**パラメータは、このテンプレートを適用する先のインデックス名を指定しています。
  - **"template" : "*" **としているので全インデックスに適用されます。
- あるインデックスが複数のテンプレートにマッチする場合、**"order"** に指定した数が大きいものが優先されます。
  - このコマンドでは **"order" : 0 ** としているので最低の優先度となっており、よってどんな追加の指定でも上書き可能です。
- **"number_of_shards"**、**"number_of_replicas" **でShard数、Replica数を指定しています。


テンプレートの変更を投入すること自体は任意のタイミングで可能ですが、変更前に投入されたインデックスに変更結果は適用されません。  
インデックスの投入前に必要なテンプレートを投入する必要があります。  
詳細はElasticsearch Referenceの[Index Templates](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-templates.html)を参照してください。

### Logstashのインストール
Logstashのパッケージをインストールします。

次のコマンドを実行します。

In [15]:
!ansible-playbook playbooks/install_logstash.yml -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [install logstash 5.0.0] **************************************************
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   



## クラスタのステータスの確認方法と監視方法
クラスタのステータスを確認するためにCluster APIを発行します。

> ※本節では、クラスタの簡単な状態を確認するAPIのみを紹介します。  
> 　詳細の確認やエラーの把握、各種リソースの状態チェックなどを実施したい場合は  
> 　**[01_05_Diagnostics.ipynb](01_05_Diagnostics.ipynb)**を利用してください。

概要を表示したい場合にはCluster Health APIを用います。  
クラスタの状態や、クラスタに属しているノードの数などを見ることができます。

In [16]:
!curl -XGET http://$ES_CLIENT/_cluster/health?pretty

{
  "cluster_name" : "es-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 3,
  "number_of_data_nodes" : 3,
  "active_primary_shards" : 0,
  "active_shards" : 0,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


上記のコマンドを実行して得られる出力の中で、**status**や**number_of_nodes**を見れば  
クラスタの状態およびクラスタ内のノード数を確認することができます。  
データ投入直後などは、クラスタの状態がyellowになっている場合があります。その際はgreenになったことを確認してから検索などを行ってください。

出力例

より詳細な情報を確認したい場合にはCluster Stats APIを用います。

In [17]:
!curl -XGET http://$ES_CLIENT/_cluster/stats?pretty

{
  "_nodes" : {
    "total" : 3,
    "successful" : 3,
    "failed" : 0
  },
  "cluster_name" : "es-cluster",
  "timestamp" : 1489648604636,
  "status" : "green",
  "indices" : {
    "count" : 0,
    "shards" : { },
    "docs" : {
      "count" : 0,
      "deleted" : 0
    },
    "store" : {
      "size_in_bytes" : 0,
      "throttle_time_in_millis" : 0
    },
    "fielddata" : {
      "memory_size_in_bytes" : 0,
      "evictions" : 0
    },
    "query_cache" : {
      "memory_size_in_bytes" : 0,
      "total_count" : 0,
      "hit_count" : 0,
      "miss_count" : 0,
      "cache_size" : 0,
      "cache_count" : 0,
      "evictions" : 0
    },
    "completion" : {
      "size_in_bytes" : 0
    },
    "segments" : {
      "count" : 0,
      "memory_in_bytes" : 0,
      "terms_memory_in_bytes" : 0,
      "stored_fields_memory_in_bytes" : 0,
      "term_vectors_memory_in_bytes" : 0,
      "norms_memory_in_bytes" : 0,
      "points_memory_in_bytes" : 0,
      "doc_values_memory_in_bytes" 

このコマンドでは、シャードやメモリに関する情報を取得することができます。  
詳細はElasticsearch Referenceの[Cluster Stats](https://www.elastic.co/guide/en/elasticsearch/reference/current/cluster-stats.html)を参照してください。

### Monitoringによる監視

Monitoring([公式ページ](https://www.elastic.co/guide/en/x-pack/current/xpack-monitoring.html))はサブスクリプションを購入すると利用できる「X-Pack」をインストールすることで有効になる機能です。  

Elasticsearchの稼働状況を監視するプラグインで、CPU使用率やディスクサイズ、Latencyなどの情報を取得することができます。  
取得した情報はElasticsearchに登録されるので、あとから遡ってクラスタのステータスを確認することも可能です。  

また、X-Packに含まれるWatcherを用いることで、特定の条件下で通知を行うこともできます。  
例） CPU使用率が特定の値を超えた、ディスク空き容量が一定量を下回ったなど。

## Logstashへのサンプルデータ投入（オプション）
インストール済みのLogstashに、この後のNotebookで利用するサンプルデータを投入します。

＃**この後の「02_Search.ipynb」以降のサンプルコマンドを実施する場合は、本節によりサンプルデータを投入してください。**  
＃**サンプルデータが不要であれば、この節の内容はスキップしてください。**

### 環境設定ファイルの設置
Logstash用の環境設定ファイルをサーバにコピーします。

サンプルデータをElasticsearchに登録するための設定は、[logstash_conf/main.conf](logstash_conf/main.conf)を用います。  
LogstashサーバがData Nodeと同じホストに無い場合は、  
main.confの "**hosts => ～～**" の部分にClient Nodeか、それが無い場合はData Nodeのリストを定義してください。  
複数の場合は  
["127.0.0.1:9200","127.0.0.2:9200"]  
のようにします。

次のリンクから修正してください。

- **[main.conf](../edit/logstash_conf/main.conf)**

ここで利用するmain.confは標準入力で読み込んだCSV形式の内容をElasticsearchに格納するための設定です。  
具体的な意味は[04_Store_Data](04_Store_Data.ipynb)で説明します。

main.confを /etc/logstash/conf.d/ 配下にコピーします。

In [18]:
!ansible-playbook playbooks/copy_logstash_conf.yml --extra-vars "notebooks_home=$notebooks_home" -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [copy local main.conf to remote] ******************************************
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   



### データ型の定義（マッピング定義）
サンプルデータ用のCSVの各列について、データ型をあらかじめ定義しておきます。

データ型の種類およびマッピング定義の詳細に関しては[05_Indexing](05_Indexing.ipynb)を参照してください。  
次のコマンドを実行してマッピング定義を行います。  

In [19]:
%%bash
curl -XPUT "http://$ES_CLIENT/_template/weather" -d @- << EOF
{
  "template" : "meteorological-data-*",
  "mappings": {
    "logs": {
      "dynamic_templates" : [
        {
          "my_strings" : {
            "match_mapping_type" : "string",
            "mapping" : {
              "type" : "keyword"
            }
          }
        }
      ],
      "properties": {
        "atmospheric_pressure": {
          "type": "float"
        },
        "sea_level_pressure":{
          "type": "float"
        },
        "precipitation_day": {
          "type": "integer"
        },
        "precipitation_day": {
          "type": "float"
        },
        "precipitation_max_hour":{
          "type":"float"
        },
        "precipitation_max_10min":{
          "type":"float"
        },
        "temperature_avg":{
          "type":"float"
        },
        "temperature_max":{
          "type":"float"
        },
        "temperature_min":{
          "type":"float"
        },
        "humidity_avg":{
          "type":"float"
        },
        "humidity_min":{
          "type":"float"
        },
        "wind_speed_avg":{
          "type":"float"
        },
        "wind_speed_max":{
          "type":"float"
        },
        "wind_speed_max_moment":{
          "type":"float"
        },
        "sunshine_duration":{
          "type":"float"
        },
        "snowfall":{
          "type":"float"
        },
        "snowfall_max":{
          "type":"float"
        }
      }
    }
  }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1394  100    21  100  1373    134   8823 --:--:-- --:--:-- --:--:--  8858


### サンプルデータの投入
実際のサンプルデータをLogstashに投入します。

ここでは気象庁から提供されている気象データを利用します。
[気象庁「過去の気象データ・ダウンロード」](http://www.data.jma.go.jp/gmd/risk/obsdl/)

公開されているデータのうち、2015年の東京のデータ(1年分)を本Notebookで利用するCSV形式に変換し、サンプルデータとして利用します([tokyo2015.csv](sample_data/tokyo2015.csv))。

サンプルデータと気象庁データの項目は、次のように対応しています。

| サンプルデータの項目 | 気象庁データの項目 | 備考 |
|-----------|--------------------|------|
| @timestamp | 年月日 ||
| location | 地点 | 気象庁データに直接の記載はないため、変換時に追加した |
| atmospheric_pressure | 日平均現地気圧 ||
| sea_level_pressure | 日平均海面気圧 ||
| precipitation_day | 降水量の日合計 ||
| precipitation_max_hour | 1時間降水量の日最大 ||
| precipitation_max_10min | 10分間降水量の日最大 ||
| temperature_avg | 日平均気温 ||
| temperature_max | 日最高気温 ||
| temperature_min | 日最低気温 ||
| humidity_avg | 日平均相対湿度 ||
| humidity_min | 日最小相対湿度 ||
| wind_speed_avg | 日平均風速 ||
| wind_speed_max | 日最大風速 ||
| wind_direction | 日最大風速（風向） ||
| wind_speed_max_moment | 日最大瞬間風速 ||
| wind_direction_max_moment | 日最大瞬間風速（風向） ||
| sunshine_duration | 日照時間 ||
| snowfall | 降雪量の日合計 ||
| snowfall_max | 日最深積雪 ||
| information_daytime | 天気概況（昼：06時～18時） ||
| information_night | 天気概況（夜：18時～翌日06時） |||

気象庁データの利用あたっては気象庁が公開している[利用規約](http://www.jma.go.jp/jma/kishou/info/coment.html)を参照してください。

登録するデータの内容は次のコマンドで確認できます。1行目がデータの項目を表し、2行目以降が実際のデータです。  
Elasticsearchは慣習として時刻情報を@timestampというフィールドに格納します。

In [20]:
!head -n 5 ./sample_data/tokyo2015.csv

Logstashのサーバーにサンプルデータをコピーします。   
Logstashをインストールしたサーバーの ~/ 配下に[sample_data](sample_data)のtokyo2015.csvをコピーします。

In [21]:
!ansible-playbook playbooks/copy_sample_data.yml --extra-vars "notebooks_home=$notebooks_home" -i ./hosts -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [setup] *******************************************************************
ok: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

TASK [copy local sample data to remote] ****************************************
changed: [ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com]

PLAY RECAP *********************************************************************
ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com : ok=2    changed=1    unreachable=0    failed=0   



Logstashを使ってサンプルデータをElasticsearchに登録します。次のセルのコマンドを実行してください。

In [22]:
!ansible logstash-server -m shell -a "cat tokyo2015.csv | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f /etc/logstash/conf.d/main.conf" -i ./hosts -u $USER --private-key=$KEYPATH

ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



データが登録されているかどうかは、次のセルを実行することで確認できます。

In [23]:
!curl -XGET http://$ES_CLIENT/meteorological-data-*/_search/?pretty

{
  "took" : 62,
  "timed_out" : false,
  "_shards" : {
    "total" : 39,
    "successful" : 39,
    "failed" : 0
  },
  "hits" : {
    "total" : 365,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.01",
        "_type" : "logs",
        "_id" : "tokyo_15-01-09",
        "_score" : 1.0,
        "_source" : {
          "date" : "15-01-09",
          "wind_speed_avg" : "4.1",
          "snowfall" : null,
          "temperature_max" : "11.0",
          "precipitation_day" : null,
          "wind_speed_max" : "8.1",
          "sunshine_duration" : "8.9",
          "temperature_avg" : "6.7",
          "temperature_min" : "3.5",
          "@version" : "1",
          "wind_direction_max_moment" : "西",
          "information_night" : "晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-01-09T00:00:00.000Z",
   

登録したデータの内容がJSON形式で表示されるので、それを確認してください。  
レスポンスのサイズを指定していないため、ここでは最大10件まで確認できます。  

### 分散配置の状態確認
投入したでデータがData Nodeにどのように配置されているか確認します。

次のコマンドで、Data NodeごとにPrimary/Replica Shardが分散配置されている状況を確認できます。

In [24]:
!curl -XGET http://$ES_CLIENT/_cat/shards?v

index                       shard prirep state   docs  store ip            node
meteorological-data-2015.05 2     r      STARTED    9 14.1kb 172.31.23.86  o4Kr9fx
meteorological-data-2015.05 2     p      STARTED    9 28.4kb 172.31.30.87  L7zNsGG
meteorological-data-2015.05 1     p      STARTED   12 24.3kb 172.31.28.229 XaJPByc
meteorological-data-2015.05 1     r      STARTED   12 16.8kb 172.31.30.87  L7zNsGG
meteorological-data-2015.05 0     p      STARTED   10 23.4kb 172.31.23.86  o4Kr9fx
meteorological-data-2015.05 0     r      STARTED   10   15kb 172.31.28.229 XaJPByc
meteorological-data-2015.04 2     r      STARTED    8 13.8kb 172.31.23.86  o4Kr9fx
meteorological-data-2015.04 2     p      STARTED    8 13.8kb 172.31.30.87  L7zNsGG
meteorological-data-2015.04 1     p      STARTED   12 16.2kb 172.31.28.229 XaJPByc
meteorological-data-2015.04 1     r      STARTED   12 16.2kb 172.31.30.87  L7zNsGG
meteorological-data-2015.04 0     p      STARTED   10 15.5kb 172.31.23.86  o4Kr9fx
meteoro

例えばShard数が3、Replica数が1の場合、あるインデックスの分の表示は次のようになります。

左列からの順に次の意味です。

|列名|意味|値の例|
|----|----|----|
|index|インデックス名|meteorological-data-2015.09|
|shard|Shard番号|1|
|prirep|Primary(p)かReplica(r)か|p|
|state|状態|STARTED|
|docs|ドキュメント数|11|
|store|ディスクサイズ|31.4kb|
|ip|配置先のData NodeのIP|172.31.27.2|


※Shardの配置のイメージについては[01_01_Outline.ipynbの「Elasticsearchクラスタ」](01_01_Outline.ipynb#Elasticsearchクラスタ)の章にある図を参照してください。

よって、次のようになっていればShard数、Replica数は正しく反映されています。
- Shard番号の数の種類がShard数と同じだけある  
（例えばShard数が3であれば、Shard番号は0,1,2の3種類あるはず）
- あるインデックスのShard番号ごとに、"r"のついた行がReplica数だけある